In [1]:
import requests #to get the url
from bs4 import BeautifulSoup #to parse html objects
import pandas as pd #to create the dataframe

In [2]:
neighborhoods_2022 = ['archipelbuurt',
 'centrum',
 'groente_en_fruitmarkt',
 'schildersbuurt',
 'stationsbuurt',
 'transvaalkwartier',
 'willemspark',
 'zeeheldenkwartier',
 'bouwlust',
 'leyenburg',
 'moerwijk',
 'morgenstond',
 'rustenburg_en_oostbroek',
 'wateringse_veld',
 'zuiderpark',
 'benoordenhout',
 'bezuidenhout',
 'mariahoeve_en_marlot',
 'binckhorst',
 'laakkwartier_en_spoorwijk',
 'forepark',
 'hoornwijk',
 'leidschenveen',
 'ypenburg',
 'bohemen_en_meer_en_bos',
 'kijkduin_en_ockenburgh',
 'kraayenstein_en_vroondaal',
 'loosduinen',
 'waldeck',
 'mobiel_stembureau',
 'belgisch_park',
 'duindorp',
 'duinoord',
 'geuzen_en_statenkwartier',
 'scheveningen',
 'van_stolkpark_en_scheveningse_bosjes',
 'westbroekpark_en_duttendel',
 'zorgvliet',
 'bomen_en_bloemenbuurt',
 'regentessekwartier',
 'valkenboskwartier',
 'vogelwijk',
 'vruchtenbuurt']

In [3]:
neighborhoods_2021 = ['archipelbuurt',
 'centrum',
 'groente_en_fruitmarkt',
 'schildersbuurt',
 'stationsbuurt',
 'transvaalkwartier',
 'willemspark',
 'zeeheldenkwartier',
 'bouwlust',
 'leyenburg',
 'moerwijk',
 'morgenstond',
 'rustenburg_en_oostbroek',
 'wateringse_veld',
 'zuiderpark',
 'benoordenhout',
 'bezuidenhout',
 'mariahoeve_en_marlot',
 'binckhorst',
 'laakkwartier_en_spoorwijk',
 'forepark',
 'hoornwijk',
 'leidschenveen',
 'ypenburg',
 'bohemen_en_meer_en_bos',
 'kijkduin_en_ockenburgh',
 'kraayenstein_en_vroondaal',
 'loosduinen',
 'waldeck',
 'mobiel_stembureau',
 'belgisch_park',
 'duindorp',
 'duinoord',
 'geuzen_en_statenkwartier',
 'scheveningen',
 'van_stolkprk_en_schevbosjes',
 'westbroekpark_en_duttendel',
 'zorgvliet',
 'bomen_en_bloemenbuurt',
 'regentessekwartier',
 'valkenboskwartier',
 'vogelwijk',
 'vruchtenbuurt']

In [4]:
pdf_2022 = pd.DataFrame()
pdf_2021 = pd.DataFrame()

In [5]:
r = requests.get('https://uitslagen.denhaag.nl/gemeenteraadsverkiezing_2022/wijken/Archipelbuurt')
soup = BeautifulSoup(r.content, "html.parser")
job_elements = soup.find_all("div", class_="result-list-page")
parties = [party.text.split(' ')[1] for party in job_elements[0].find_all("td", class_="party")]
votes = [vote.text for vote in job_elements[0].find_all("td", class_="votes")]
percentage = [float(percentage.text.replace(',', '.').replace('%', '')) for percentage in job_elements[0].find_all("td", class_="percentage")]

In [23]:
def get_votes_per_election(verkiezings_url, neighborhoods):
    '''
    input: second part of url that contains information on the elections as a string(found on https://uitslagen.denhaag.nl/gemeenteraadsverkiezing_2022/archief), e.g.'tweede-kamerverkiezing-2021' and the neighborhouds that are linked in that election. Small differences occur here, e.g.  van_stolkprk_en_schevbosjes instead of van_stolkpark_en_scheveningse_bosjes
    output: pandas dataframe with the respective party, amount of votes and the percentage of votes per neighborhood

    '''
    pdf = pd.DataFrame()
    parties = []
    votes = []
    percentage = []
    neighborhood_for_df = []
    for neighborhood in neighborhoods:
        try:
            r = requests.get(f'https://uitslagen.denhaag.nl/{verkiezings_url}/wijken/{neighborhood}')
            soup = BeautifulSoup(r.content, "html.parser")
            job_elements = soup.find_all("div", class_="result-list-page")
            parties.extend(party.text.split(' ')[1] for party in job_elements[0].find_all("td", class_="party"))
            votes.extend(vote.text for vote in job_elements[0].find_all("td", class_="votes"))
            percentage.extend(float(percentage.text.replace(',', '.').replace('%', '')) for percentage in job_elements[0].find_all("td", class_="percentage"))
            neighborhood_for_df.extend([neighborhood] * len(list(party.text.split(' ')[1] for party in job_elements[0].find_all("td", class_="party"))))
        except:
            print(f'Error: the following neighborhood is not an url: {neighborhood}')
            return None
    votes_replaced = [int(vote.replace('.','')) for vote in votes]#prevent . from turning into decimal votes 
    pdf['parties'] = parties
    pdf['votes'] = votes_replaced
    pdf['percentage'] = percentage
    pdf['neighborhood'] = neighborhood_for_df
    return pdf

In [25]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pdf_2021)

            parties  votes  percentage                 neighborhood
0               D66    430        26.5                archipelbuurt
1               VVD    420        25.9                archipelbuurt
2            Partij    140         8.6                archipelbuurt
3              Volt    106         6.5                archipelbuurt
4        GROENLINKS     90         5.5                archipelbuurt
5               CDA     83         5.1                archipelbuurt
6          P.v.d.A.     72         4.4                archipelbuurt
7               PVV     71         4.4                archipelbuurt
8             Forum     59         3.6                archipelbuurt
9                SP     31         1.9                archipelbuurt
10             JA21     28         1.7                archipelbuurt
11     ChristenUnie     23         1.4                archipelbuurt
12             BIJ1     13         0.8                archipelbuurt
13             DENK     12         0.7          

In [27]:
pdf_2021 = get_votes_per_election('tweede-kamerverkiezing-2021',neighborhoods_2021)
pdf_2022 = get_votes_per_election('gemeenteraadsverkiezing_2022', neighborhoods_2022)

In [26]:
pdf_2021.to_csv('../data/raw/uitslag_2021_landelijk.csv')
pdf_2022.to_csv('../data/raw/uitslag_2022_gemeenteraad.csv')